In [6]:
import pandas as pd
import sqlite3

con = sqlite3.connect('./datasets/persons.db')

df = pd.read_sql('SELECT * FROM persons LEFT JOIN haircuts ON persons.national_number = haircuts.national_number', con)
df

,first_name,last_name,national_number,birthdate,age,gender,age_category,company,full_name,national_number,haircut,white_hair_count
0,Édouard,Hoarau,57042565594,1957-04-25,66,Male,47-113 Senior,Huet s a s,Édouard Hoarau,5.704257e+10,Chignon,458
1,Nath,Raymond,66072176208,1966-07-21,57,Female,47-113 Senior,Gimenez,Nath Raymond,NaN,None,None
2,Élise,Reynaud,80072229956,1980-07-22,43,Male,43-46 À point,Thibault boyer s a s,Élise Reynaud,8.007223e+10,Pixie cut,119
3,Laure,Gérard,89040194445,1989-04-01,34,Female,0-38 Jeune,Weiss,Laure Gérard,8.904019e+10,Chignon,298
4,Célina,Ledoux,58051144246,1958-05-11,65,Female,47-113 Senior,Blanc,Célina Ledoux,5.805114e+10,Bouffant,35
...,...,...,...,...,...,...,...,...,...,...,...,...
995,Arthur,Clerc,61081891128,1961-08-18,62,Male,47-113 Senior,Boutin sa,Arthur Clerc,6.108189e+10,Dreadlocks,345
996,Aimée,Collin,47050599541,1947-05-05,76,Male,47-113 Senior,Dias peron sa,Aimée Collin,4.705060e+10,Butch cut,492
997,Olivie,Bazin,89020852065,1989-02-08,35,Female,0-38 Jeune,Dijoux s a r l,Olivie Bazin,8.902085e+10,Pompadour,155
998,Patricia,Lucas,82102872289,1982-10-28,41,Female,39-42 Adulte,Marechal,Patricia Lucas,NaN,None,None


In [15]:
# count the number of persons by gender and haircuts
pivot_table = df.groupby(['haircut', 'gender']).size().reset_index(name='count')
pivot_table.sort_values(by='count', ascending=True)

,haircut,gender,count
29,Pompadour,Male,8
9,Bun,Male,10
25,Mullet,Male,11
6,Bowl cut,Female,12
8,Bun,Female,12
20,Hi-top fade,Female,12
15,Chonmage,Male,12
14,Chonmage,Female,12
0,Big hair,Female,13
1,Big hair,Male,13


In [16]:
# ------------------
# anonymisation functions
# ------------------

def get_first_line(df):
    df = df.sort_values(by=['count'], ascending=True)
    
    return df.iloc[0]

def check_if_need_to_be_anonymised(df, nMax):
    if df['count'].min() < nMax:
        return True
    else:
        return False

def get_similarity_score(df):
    first_line = get_first_line(df)
    
    # columns except count and score
    columns_except_count = df.columns.drop(['count'])
    if 'score' in columns_except_count:
        columns_except_count = columns_except_count.drop(['score'])

    for index, row in df.iterrows():
        score = 0
        
        for column in columns_except_count:
            if row[column] == first_line[column]:
                score += 1

        score = score / len(columns_except_count)

        # add score to df
        df.loc[index, 'score'] = score

    df = df.sort_values(by=['score', 'count'], ascending=[False, True])
    df = df.reset_index(drop=True)

    return df

def combine_similar_rows(df, count_column='count'):

    # combine each features of the two rows
    for column in df.columns:
        if column == count_column:
            df.loc[0, column] = df.iloc[0][column] + df.iloc[1][column]
        else:
            # check if the two rows have the same value
            if df.iloc[0][column] != df.iloc[1][column]:
                new_value = str(df.iloc[0][column]) + " + " + str(df.iloc[1][column])
                
                # split new_value with " + ", order and remove duplicates
                new_value = sorted(set(new_value.split(" + ")))
                new_value = " + ".join(new_value)

                df.loc[0, column] = new_value 

    df = df.drop(columns=['score'])

    # drop iloc[1] row
    df = df.drop(df.index[1])

    df = df.sort_values(by=['count'], ascending=True)
    df = df.reset_index(drop=True)

    return df

def need_to_anonymise(df, nMax):
    if df['count'].min() < nMax:
        return True

def anonymise_microdata(df, nMax):
    while check_if_need_to_be_anonymised(df, nMax) == True:
        df_with_score = get_similarity_score(df)
        df = combine_similar_rows(df_with_score)
    
    return df

In [25]:
df_anon = anonymise_microdata(pivot_table, 60)
df_anon

,haircut,gender,count
0,Butch cut + devilock,Female + Male,64
1,Bob cut + Bun + Devilock + Dreadlocks + Pompadour,Male,77
2,Devilock + Pixie cut + Pompadour + Tonsure,Female,77
3,Big hair + Bowl cut + Chonmage + Mullet + Tons...,Male,93
4,Bowl cut + Bun + Chignon + Chonmage + Hi-top f...,Female,93
5,Big hair + Bob cut + Bouffant + Dreadlocks + L...,Female,108
6,Bouffant + Chignon + Hi-top fade + Layered hai...,Male,126
